In [1]:
import numpy as np
import cv2
import os
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras_vggface.vggface import VGGFace

2023-03-09 18:46:38.596452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TRAIN_DIR = '/coding-drive/DATASETS/face-recognition-data/train'
VAL_DIR = '/coding-drive/DATASETS/face-recognition-data/test'


In [3]:
# Define the number of classes (i.e. number of people) in the training dataset
NUM_CLASSES = len(os.listdir(TRAIN_DIR))

# Define the input shape for the VGG Face model
INPUT_SHAPE = (224, 224, 3)

# Define the batch size for training and validation data
BATCH_SIZE = 32

# Define the number of epochs for training
NUM_EPOCHS = 10


In [4]:
# Instantiate the VGG Face model and remove the last layer
vggface = VGGFace(model='resnet50', include_top=False, input_shape=INPUT_SHAPE)
last_layer = vggface.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(512, activation='relu', name='fc6')(x)
out = Dense(NUM_CLASSES, activation='softmax', name='fc8')(x)
custom_vggface = Model(vggface.input, out)


2023-03-09 18:46:40.606728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 18:46:41.019843: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4646 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5


In [ ]:
# Freeze the layers of the VGG Face model except the last two
for layer in custom_vggface.layers[:-2]:
    layer.trainable = False

# Compile the model with categorical crossentropy loss and Adam optimizer
custom_vggface.compile(loss='categorical_crossentropy',
                       optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define the data generator for training data
train_datagen = ImageDataGenerator(rescale=1./255)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# Load the training data using the data generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Define the data generator for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the validation data using the data generator
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')


Found 954 images belonging to 1 classes.
Found 477 images belonging to 1 classes.


In [ ]:
# Train the model
custom_vggface.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=val_generator)

Epoch 1/10


2023-03-09 18:39:11.009746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


30/30 [==============================] - 10s 188ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
30/30 [==============================] - 4s 127ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
30/30 [==============================] - 4s 126ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
30/30 [==============================] - 4s 126ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
30/30 [==============================] - 4s 127ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
30/30 [==============================] - 4s 126ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 4s 126ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_

In [ ]:
# Save the model
custom_vggface.save('training_output/face_recognition_vggface.h5')
